Import necessary packages

In [91]:
import requests
import json

Gets the file from the Indico address of your conference. Must insert correct web address.

In [92]:
params = (
    ('detail', 'contributions'),
)

response = requests.get('https://indico.cern.ch/export/event/867789.json', params=params)

Gives JSON file of response

In [93]:
Jsonresponse = response.json()

Get into the subfolder containing all events

In [94]:
Jsonresponse=Jsonresponse["results"][0]["contributions"]

Get the event with the correct map id. This must be changed so the user can get their correct metadata

In [95]:
result=None

for item in Jsonresponse:
    result=item
    if result["id"]=="409":
        break;

Find the name of the file and the path to download the file

In [96]:
data_file_name=result["folders"][0]["attachments"][0]["filename"]
file_download_path=result["folders"][0]["attachments"][0]["download_url"]

User input, specify which upload type you want to define your upload as. Poster, presentation, software, etc. Also insert your Zenodo Access Token.

In [97]:
Upload_type="nan"
Access_Token="Something"

Get description of project

In [98]:
description=result["description"]

Get title

In [99]:
Title=result["title"]

Get author name and affiliations

In [100]:
Author_Names=result["primaryauthors"][0]["fullName"]
Author_Affiliation=result["primaryauthors"][0]["affiliation"]

Ensure metdata is correct type

In [101]:
if type(data_file_names)!=str:
    print("file name must be string")
if type(file_download_path)!=str:
    print("file download path must be string")
if type(description)!=str:
    print("description must be string")
if type(Upload_type)!=str:
    print("upload type must be string")
if type(Access_Token)!=str:
    print("file name must be string")

Create empty Zenodo upload

In [102]:
headers = {"Content-Type": "application/json"}

r=requests.post('https://sandbox.zenodo.org/api/deposit/depositions', 
                params={'access_token': Access_Token}, json={},headers=headers)

Get the id for the entry just created so it can be edited

In [103]:
deposition_id= r.json()['id']

KeyError: 'id'

Put file into the Zenodo upload

In [104]:
data={'name': data_file_name}

files={'file': open(file_download_path, 'rb')}

r=requests.post('https://sandbox.zenodo.org/api/deposit/depositions/%s/files' % deposition_id, 
                params={'access_token': Access_Token}, data=data, files=files)

FileNotFoundError: [Errno 2] No such file or directory: 'https://indico.cern.ch/event/867789/contributions/3754987/attachments/1989047/3315426/Poster_HSFWLCG.pdf'

All the metadata you want to put in the file

In [105]:
data={ 'metadata': {'title': Title, 'upload_type': Upload_type, 'description': description,
                    'creators': [{'name': Author_Names, 'affiliation': Author_Affiliation}]}}

Input all metadata in Zenodo upload

In [106]:
r=requests.put('https://sandbox.zenodo.org/api/deposit/depositions/%s' % deposition_id, 
               params={'access_token': Access_Token}, data=json.dumps(data), headers=headers)